<a href="https://colab.research.google.com/github/tobiashomie/image-detection/blob/main/TL_VGG_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [4]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from pickle import dump
from io import BytesIO
import urllib

# Utility methods

In [5]:
def load_img_from_url(URL, target_size):
    with urllib.request.urlopen(URL) as url:
        img = load_img(BytesIO(url.read()), target_size=target_size)

    return img_to_array(img)

# Constants

In [6]:
image_url = "https://raw.githubusercontent.com/tobiashomie/image-detection/main/mug.png"
target_size = (224, 224)

# Pre-trained model as a classifier

## Import image and preprocess

In [7]:
# load an image from file
image_raw = load_img_from_url(URL=image_url, target_size=target_size)
# convert the image pixels to a numpy array
image_array = img_to_array(image_raw)
# reshape data for the model
image_reshaped = image_array.reshape((1, image_array.shape[0], image_array.shape[1], image_array.shape[2]))
# prepare the image for the VGG model
image_for_model = preprocess_input(image_reshaped)

## Load VGG model and predict

In [8]:
model = VGG16()
yhat = model.predict(image_for_model)
labels = decode_predictions(yhat)
print("labels: ", labels)
print("prediction result: ", labels[0][0])

35363/35363 [==============================] - 0s 0us/step
labels:  [[('n03063599', 'coffee_mug', 0.7043004), ('n03063689', 'coffeepot', 0.10230983), ('n07930864', 'cup', 0.075528815), ('n04398044', 'teapot', 0.03837995), ('n03950228', 'pitcher', 0.030945685)]]
prediction result:  ('n03063599', 'coffee_mug', 0.7043004)
